# Исследование надежности заемщиков.

#### Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.<br>
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.<br>

### Шаг 1. Обзор данных

**Описание данных:**
- `children` — количество детей в семье 
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

In [1]:
import pandas as pd
try:
    df = pd.read_csv('/datasets/data.csv')
except:
    df = pd.read_csv('data.csv')
df.head() #добавлено для корректного вывода

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
df.info() # получение информации о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [4]:
df['days_employed'].min()

-18388.949900568383

Столбцы `days_employed` и `total_income` имеют пропуски, количество строк с пропусками 2174, около 10% всей выборки.<br>
Имеются отрицательные значение общего трудового стажа в днях и количества детей.<br>
Так же имеются семьи с 20 детьми, при отсутствуют семьи с детьми больше 5 и меньше 20, значит это тоже ошибка.

### Шаг 2.1 Проверка данных на аномалии и исправления. 

In [5]:
#подсчет количества ячеек с отрицательными значениями трудового стажа
display(df[df['days_employed'] < 0]['gender'].count())
#проверка зависимости наличия отриццательных значений общего трудового стажа от цели кредита
df[df['days_employed'] < 0].groupby('purpose')['gender'].count()

15906

purpose
автомобили                                340
автомобиль                                363
высшее образование                        342
дополнительное образование                332
жилье                                     485
заняться высшим образованием              364
заняться образованием                     298
на покупку автомобиля                     357
на покупку подержанного автомобиля        353
на покупку своего автомобиля              382
на проведение свадьбы                     559
недвижимость                              480
образование                               331
операции с жильем                         483
операции с коммерческой недвижимостью     474
операции с недвижимостью                  502
операции со своей недвижимостью           462
покупка жилой недвижимости                461
покупка жилья                             483
покупка жилья для сдачи                   486
покупка жилья для семьи                   463
покупка коммерческой недви

Отрицательных значений трудового стажа 15906, более половины всех записей.<br>

Отрицательные значения в трудовом стаже не зависят от цели получения кредита.<br>

С учетом отсутсвия связи между отрицательными значениями трудового стажа и целей кредита, куда не входят цели условного выживания от потери работы, принимаем **причиной данных значений ошибку в <u>заполнении данных</u>.**<br>

Для корректного заполнения пропусков в таблице исправим данную ошибку.

In [6]:
#заменим отрицательные значения в столбце с трудовым стажем
df['days_employed'] = df['days_employed'].abs()
df['days_employed'].min()

24.14163324048118

Количество детей отрицательным быть не может, похоже на техническую ошибку, необходимо скорректировать данные.

In [7]:
#замена отрицательных значений в столбце с количеством детей
df['children'] = df['children'].abs()
df['children'].min()

0

Как выяснили ранее, 20 детей тоже ошибка, в связи с этим исправим данную ошибку в датафрейме.

In [8]:
#заменим количество детей 20 на 2
df['children'] = df['children'].replace(20, 2)
df['children'].max()

5

In [9]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [10]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В датасете так же есть ошибка в стобце возраст, равный нулю, и неопределенность в столбце пол = XNA.

In [11]:
df['dob_years'].std() #вычислим стандартное отклонение для столбца возраст

12.574583857924386

Среднеквадратичное отклонение равно 12, данные по возврасту можно заменить средними значениями.

In [12]:
df.loc[(df['dob_years']==0) , 'dob_years'] = df['dob_years'].mean().astype('int')
df['dob_years'].value_counts()

35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Касательно гендера - статистически, клиентов женского пола больше, поэтому выполним замену ошибки женский.

In [13]:
df.loc[(df['gender']=='XNA') , 'gender'] = 'F'
df['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

### Шаг 2.2 Заполнение пропусков.

**Выполним проверку зависимости пропущенных и аномальных данных в зависимости от типа занятости.**

In [14]:
#проверка совпадения ячеек с пропусками в двух столбцах
display(df[df['total_income'].isna()]['days_employed'].count())
#проверка зависимости наличия пропущенных ячеек от типа занятости
df[df['total_income'].isna()].groupby('income_type')['gender'].count()

0

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: gender, dtype: int64

Строки с пропущенными ячейками в столбцах `days_employed` и `total_income` совпадают.<br>
*Пропущенные данные не зависят от типа занятости.*

In [15]:
#расчет среднеквадратических отклонений для столбцов с пропущенными ячейкми
display(df['days_employed'].std()) 
df['total_income'].std()

139030.88052749448

102971.56644797762

Среднеквадратическия отклонения >100000, значит значения в данных колонках сильно расходятся со средним значением, поэтому для заполнения пропусков более корректно будет использовать **медиану**.

In [16]:
#заменим пропущенные значения на медианы
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
df['total_income'] = df['total_income'].fillna(df['total_income'].median())
#проверим информацию о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.3. Изменение типов данных.

Столбцы `days_employed` и `total_income` имеют формат `float`, при это при расчете потенциального кредитного рейтигнга часы работы и копейки в зарплате не представляют ценности.<br> 
Для удобства и оптимизации заменим тип данных на целые числа.

In [17]:
#перевод типов значений в столбцах days_employed и total_income
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Для получения корректных данных необходимо очистить таблицу от дубликатов.
В столбце `education` имеются ячейки как со строчными, так и прописными буквами, предварительно отформатируем данную колонку


In [18]:
#приведем все ячейки столбца education к строчному формату
df['education'] = df['education'].str.lower()
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [19]:
#удаление дубликатов из датафрейфма
df = df.drop_duplicates().reset_index(drop=True)

Проверим на предмет неявных дубликатов столбец `family_status`

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Определим какие значения соответсвтуют идентификаторам семейного положения и уровня образования.

In [20]:
display(df.groupby('education').agg({'education_id': ['min', 'max']}))
df.groupby('family_status').agg({'family_status_id': ['min', 'max']})

education_id    
                             min max
education                           
высшее                         0   0
начальное                      3   3
неоконченное высшее            2   2
среднее                        1   1
ученая степень                 4   4

family_status_id    
                                   min max
family_status                             
Не женат / не замужем                4   4
в разводе                            3   3
вдовец / вдова                       2   2
гражданский брак                     1   1
женат / замужем                      0   0

Подготовим "словари" с идентификатором и соответствующим ему значением в столбцах семейное положение и уровень образования.

In [21]:
#создадим "словари" с идентификатором и соответствующим ему значением в столбцах семейное положение и уровень образования
education_dictionary = df[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
family_status_dictionary = df[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
display(education_dictionary)
family_status_dictionary

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [22]:
#удалим стобцы education и family_status для удобства последующего анализа
df = df.drop(columns=['education', 'family_status'])

### Шаг 2.6. Категоризация дохода.

Для упрощения анализа категоризируем клиентов по уровню дохода:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

In [23]:
#создадим функцию для категоризации клиентов по уровню дохода
def income_category(income):
    if income <= 30000:
        return 'E'
    if 30000 < income <= 50000:
        return 'D'
    if 50000 < income <= 200000:
        return 'C'
    if 200000 < income <= 1000000:
        return 'B'
    return 'A'
df['income_category'] = df['total_income'].apply(income_category)
df.groupby('income_category')['total_income'].count()

income_category
A       25
B     5041
C    16016
D      350
E       22
Name: total_income, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

Так же для упрощения анализа категоризируем клиентов по целям кредита:
- операции с автомобилем
- операции с недвижимостью
- проведение свадьбы
- получение образования

In [24]:
#создадим функцию для категоризации клиентов по уровню дохода
def purpose_category(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'образов' in purpose:
        return 'получение образования'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if ('недвиж' in purpose) or ('жиль' in purpose):
        return 'операции с недвижимостью'
    return 'ошибка'
df['purpose_category'] = df['purpose'].apply(purpose_category)
df.groupby('purpose_category')['purpose'].count()

purpose_category
операции с автомобилем       4306
операции с недвижимостью    10811
получение образования        4013
проведение свадьбы           2324
Name: purpose, dtype: int64

### Ответы на вопросы.

##### Вопрос 1:
Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [25]:
#определим долю невозвратов кредитов в зависимости от количества детей
df.groupby('children')['debt'].agg('mean')*100

children
0    7.543822
1    9.165808
2    9.492481
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64

##### Вывод 1:
Наименьший процент невозвратов кредитов в срок у людей, у которых нет детей. <br>
При количестве детей от 1 до 4 кореляции минимальны.<br>
При количестве 5 нет просрочек, но, скорее всего, это связано с отсутствием значимого количества кейсов для семей с 5 детьми.

**Наибольший процент просрочех у семей с детьми связан с тем, что дети являются постоянным источником непредвиденных расходов, особенно говоря о здоровье. Исходя из полученных данных, более высокий ретинг следует давать семьям без детей.**

##### Вопрос 2:
Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
family_status_dictionary

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [27]:
#определим долю невозвратов кредитов в зависимости от количества детей
df.groupby('family_status_id')['debt'].agg('mean')*100

family_status_id
0    7.545182
1    9.347145
2    6.569343
3    7.112971
4    9.750890
Name: debt, dtype: float64

##### Вывод 2:
Наименьший процент невозвратов в срок - у овдовевших клиентов.<br>
Так же женаты и в разводе имеют более низкие значения просрочек.<br>
Наибольший процент невозвратов в срок имеют клиенты не женатые, либо находящиеся в гражднаском браке.

**-Статистика показывает, что не женатые/не замужние и находящиеся в гражданоском браке (категории в целом можно приравнять), должны иметь более низкий рейтинг при расчете кредитного рейтинга. <br>
-Данная ситуация может быть связана с отсутствием совместного бюджета, если говорить о гражданском браке, когда любые непредвиденные расходы или отсутствие поступления денег, не перекрываются деньгами партнера.<br>
-У неженатных процент невозвратов может быть выше в связи с тем, что они, скорее всего, моложе, находятся в процессе обучения и не всегда имеют стабильный источник дохода.** 

##### Вопрос 3:
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для упрощения анализа категоризируем клиентов по уровню дохода:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

In [28]:
#определим долю невозвратов кредитов в зависимости от количества детей
df.groupby('income_category')['debt'].agg('mean')*100

income_category
A    8.000000
B    7.062091
C    8.491508
D    6.000000
E    9.090909
Name: debt, dtype: float64

##### Вывод 3:
Наименьший процент невозвратов в срок - у клиентов со средним уровнем дохода (категория D).<br>
Клиенты с доходом 200,001 - 1,000,000 так же имеют довольно низкие показатели просрочек.<br>
Высокие проценты невыплат кредитов в срок клиентов категии С и А.<br>
Максимальный процент невозвратов у клиентов доходом <30,000 рублей.

**Общей кореляции нет, однако клиенты с доходом ниже среднего в России имеют в целом более высокий процент невозврата, им следуют давать более низкую оценку в кредитном рейтинге.**

##### Вопрос 4:
Как разные цели кредита влияют на его возврат в срок

In [29]:
#определим долю невозвратов кредитов в зависимости от количества детей
df.groupby('purpose_category')['debt'].agg('mean')*100

purpose_category
операции с автомобилем      9.359034
операции с недвижимостью    7.233373
получение образования       9.220035
проведение свадьбы          8.003442
Name: debt, dtype: float64

##### Вывод 4:
Наименьший процент невозвратов у операций связанных с недвижимостью.<br>
Операции с автомобилем и кредит на образование имеют наибольшую долю невыплат в срок. <br>

**-Операции с недвижимостью имеют наибольший процент возвратов в связи с особенностями получения ипотечных кредитов - необходимо певрный взнос, больше проверок и в целом, более осознанная покупка.<br>
-Меньший процент возратов по кредитам с автомобилем связан с тем, что часто - это траты на инструменты производства, которые имеют свойство ломаться, из-за чего простои приводят к невозвратами в срок.<br>
-Просрочки по кредитам на образованием связаны с тем, что люди которые берут данный кредит, либо на данный момент не имеют постоянного источника дохода, либо в начале карьерного пути, в связи с этим сложнее сформировать подушку безопасности, которая позволит скомпенсировать непредвиденные расходы.<br>
-Более высокий рейтинг следуют дать операциям с недвижимостью, низкий - на операции с автомобилем и получение образования.**

## Общий вывод:

Перед подведением общих итогов подготовим сводные таблицы по проценту невозвратов в завимимости от количества детей и семейного положения.

In [30]:
family_status_dictionary

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [31]:
(df.pivot_table(index=['children'],columns=['family_status_id'],
values ='debt',aggfunc='sum'))/(df.pivot_table(index=['children'],
columns=['family_status_id'],values ='debt',aggfunc='count'))*100

family_status_id,0,1,2,3,4
children,,,,,
0,6.909480,8.388278,6.257379,7.015306,9.283820
1,8.222370,11.800000,8.641975,6.645570,11.453744
2,9.355247,9.295775,12.500000,9.638554,11.904762
3,6.827309,14.285714,0.000000,9.090909,12.500000
4,10.344828,0.000000,0.000000,0.000000,50.000000
5,0.000000,0.000000,NaN,NaN,NaN


**Отвечая на вопрос проекта:** красный флаг - это не замужние / не женатые клиенты с 4 детьми. <br>
Наличие хотя бы одного ребенка резко снижеает процент возвратов в срок у не замужних / не женатых клиентов, включая аналогичную категорию, находящихся в гражданском браке.<br>
Как указывал ранее, скорее всего, это связано с тем, что все финансовые риски ложаться на 1 человека, в то же время ребенок - сам по себе риск.<br>
Данные факты следует учитывать при построении кредитного рейтинга: наличие ребенка - риск, не состоящие в браке (либо состоящие в гражданском браке) - риск, совмещение факторов приводит к повышению рисков.<br>
**В дополнение:**
Исходя из статистики возвратов кредитов в зависимости от целей - более высокий рейтинг следуют дать операциям с недвижимостью, низкий - на операции с автомобилем и получение образования.<br>
Более низкий рейтинг необходимо дать категории клиентов, у которых доход ниже среднестатистического дохода по России.<br>
**Касательно полученного датафрейма:**
Были устранены пропуски, ошибки, аномалии данных практически в каждом из столбцов полученного датасета, для оптимизации работы необходимо ввдеение предварительной проверки введенных данных на предмет аномалий:<br>
-доход не должен быть отрицательным;<br>
-возраст клиентов ниже возраста совершеннолетия;<br>
-пол сделать только выбираемым из представленных значений;<br>
-не допускать пропуски данных.<br>